# Intro

Welcome to the interactive labeling notebook!

The notebook proceeds as follows:

1. **Import** libraries
2. Define **paths** to data
3. **Label** ROI images using a simple GUI
4. **Visualize** results
5. **Save** results


- If you need to restart a session, don't worry. Your results are continuously saved in the CSV file and you can just start from where you left off.
- We recommend labeling one session at a time so that the results can be saved with the data.

# Import libraries

Widen the notebook

In [1]:
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

Import basic libraries

In [2]:
from pathlib import Path
import random

import numpy as np
import matplotlib.pyplot as plt

Import `roicat`

In [3]:
import roicat

In [4]:
# Import vrAnalysis related code
from vrAnalysis import database

# Find paths to data

##### Prepare list of paths to data

In this example we are using suite2p output files, but other data types can be used (CaImAn, etc.) \
See the notebook on ingesting diverse data: https://github.com/RichieHakim/ROICaT/blob/main/notebooks/jupyter/other/demo_data_importing.ipynb

Make a list containing the paths to all the input files.

In this example we are using suite2p, so the following are defined:
1. `paths_allStat`: a list to all the stat.npy files
2. `paths_allOps`: a list with ops.npy files that correspond 1-to-1 with the stat.npy files

In [5]:
# Create link to vrDatabase containing all data and get session paths for target mouse
mouseName = "ATL027"
planeName = "plane0"
vrdb = database.vrDatabase('vrSessions')
ises = vrdb.iterSessions(imaging=True, mouseName=mouseName, dontTrack=False)
pathList = []
for ses in ises:
    pathList.append(ses.sessionPath())
for idx, pl in enumerate(pathList): print(idx, pl)

0 C:\Users\andrew\Documents\localData\ATL027\2023-07-19\701
1 C:\Users\andrew\Documents\localData\ATL027\2023-07-20\701
2 C:\Users\andrew\Documents\localData\ATL027\2023-07-21\701
3 C:\Users\andrew\Documents\localData\ATL027\2023-07-24\701
4 C:\Users\andrew\Documents\localData\ATL027\2023-07-25\701
5 C:\Users\andrew\Documents\localData\ATL027\2023-07-26\701
6 C:\Users\andrew\Documents\localData\ATL027\2023-07-27\701
7 C:\Users\andrew\Documents\localData\ATL027\2023-07-28\701
8 C:\Users\andrew\Documents\localData\ATL027\2023-08-01\701
9 C:\Users\andrew\Documents\localData\ATL027\2023-08-02\701
10 C:\Users\andrew\Documents\localData\ATL027\2023-08-04\702
11 C:\Users\andrew\Documents\localData\ATL027\2023-08-07\701
12 C:\Users\andrew\Documents\localData\ATL027\2023-08-08\701
13 C:\Users\andrew\Documents\localData\ATL027\2023-08-09\701


In [7]:
dir_allOuterFolders = pathList[10] #r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403L/stat_and_ops'

pathSuffixToStat = 'stat.npy'
pathSuffixToOps = 'ops.npy'

paths_stat = roicat.helpers.find_paths(
    dir_outer=dir_allOuterFolders,
    reMatch=pathSuffixToStat,
    depth=6,
)[:]
paths_ops  = np.array([Path(path).resolve().parent / pathSuffixToOps for path in paths_stat])[:]

print(f'paths to all stat files:');
[print(path) for path in paths_stat];
print('');
print(f'paths to all ops files:');
[print(path) for path in paths_ops];

paths to all stat files:
C:\Users\andrew\Documents\localData\ATL027\2023-08-04\702\suite2p\plane0\stat.npy
C:\Users\andrew\Documents\localData\ATL027\2023-08-04\702\suite2p\plane1\stat.npy
C:\Users\andrew\Documents\localData\ATL027\2023-08-04\702\suite2p\plane2\stat.npy
C:\Users\andrew\Documents\localData\ATL027\2023-08-04\702\suite2p\plane3\stat.npy
C:\Users\andrew\Documents\localData\ATL027\2023-08-04\702\suite2p\plane4\stat.npy

paths to all ops files:
C:\Users\Andrew\Documents\localData\ATL027\2023-08-04\702\suite2p\plane0\ops.npy
C:\Users\Andrew\Documents\localData\ATL027\2023-08-04\702\suite2p\plane1\ops.npy
C:\Users\Andrew\Documents\localData\ATL027\2023-08-04\702\suite2p\plane2\ops.npy
C:\Users\Andrew\Documents\localData\ATL027\2023-08-04\702\suite2p\plane3\ops.npy
C:\Users\Andrew\Documents\localData\ATL027\2023-08-04\702\suite2p\plane4\ops.npy


In [8]:
data = roicat.data_importing.Data_suite2p(
    paths_statFiles=paths_stat,
    paths_opsFiles=paths_ops,
    new_or_old_suite2p='new',
    verbose=True,
)

assert data.check_completeness(verbose=False)['classification_inference'], f"Data object is missing attributes necessary for tracking."

c:\users\andrew\documents\github\roicat\roicat\util.py:416: UserWarning: RH WARNING: unable to get gcc version. Got error: [WinError 2] The system cannot find the file specified
  warnings.warn(f'RH WARNING: unable to get gcc version. Got error: {e}')


Starting: Importing FOV images from ops files
Completed: Set FOV_height and FOV_width successfully.
Completed: Imported 5 FOV images.
Setting FOV_images...
Completed: Set FOV_images for 5 sessions successfully.
Importing spatial footprints from stat files.


100%|██████████| 5/5 [00:02<00:00,  1.74it/s]


Imported 5 sessions of spatial footprints into sparse arrays.
Completed: Set spatialFootprints for 5 sessions successfully.
Completed: Created session_bool.
Completed: Created centroids.
Starting: Creating centered ROI images from spatial footprints...
Completed: Created ROI images.


# Label data

How to use this GUI:
- Each image will be shown one at a time. You can go to previous and next images by using the ***Left*** and ***Right*** arrow keys, and can close the GUI by pressing the ***Escape*** key.
- Label the image by pressing any key on your keyboard. This key will be used as the label. After each key press, the following occurs:
    1. The results will be saved as a CSV file at `path_csv` (will be a temporary file if not specified)
    2. The results will also be stored in the python object as `ImageLabeler_obj.labels_`.
    2. The GUI should progress to the next image.
    3. Finish labeling by closing out of the GUI window or pressing the user defined '`end_key`' (***Escape*** by default).
- The results are stored in a temporary CSV file with two columns: 'index' and 'label'. This file can be used to recover lost labeling session data. We will save more comprehensive results files at the end of the notebook.
    - **'index'**: The index within the data object. 0-indexed.
    - **'label'**: The key pressed for that image in the labeling GUI.
- To start at a different image index than 0, change the `start_index` argument.

In [11]:
labels.keys()

dict_keys(['labels_list', 'labels_dict', 'labels_df'])

In [10]:
images = np.concatenate(data.ROI_images, axis=0)

## Run the ImageLabeler class
with roicat.helpers.ImageLabeler(
    image_array=images, 
    start_index=0,  ## Change the starting image index here.
    path_csv=None,  ## If None, then a temporary path will be used.
    resize_factor=10.0,  
    verbose=True,
    key_end='Escape',  ## Closes GUI
    key_prev='Left',  ## Goes to previous image
    key_next='Right',  ## Goes to next image
) as labeler:  ## Running this object within a context manager allows it to crash gracefully
    labeler.run()
    
labels = {
    'labels_list': labeler.get_labels(kind='list'),
    'labels_dict': labeler.get_labels(kind='dict'),
    'labels_df': labeler.get_labels(kind='dataframe'),
}

print(f"Number of labeled ROIs: {len(labels['labels_list'])}")
print(f"path to csv file: {labeler.path_csv}")

Image 0: c
Image 1: c
Image 2: c
Image 3: c
Image 4: c
Image 5: c
Image 6: c
Image 7: c
Image 8: c
Image 9: c
Image 10: c
Image 11: c
Image 12: c
Image 13: c
Image 14: c
Image 15: c
Image 16: c
Image 17: c
Image 18: c
Image 19: c
Image 20: c
Image 21: c
Image 22: c
Image 23: c
Image 24: c
Image 25: w
Image 26: c
Image 27: c
Image 28: c
Image 29: c
Image 30: c
Image 31: c
Image 32: c
Image 33: c
Image 34: c
Image 35: c
Image 36: c
Image 37: c
Image 38: c
Image 37: w
Image 38: c
Image 39: c
Image 40: c
Image 41: o
Image 42: c
Image 43: c
Image 44: c
Image 45: c
Image 46: c
Image 47: c
Image 48: c
Image 49: c
Image 50: c
Image 51: c
Image 52: c
Image 53: c
Image 54: c
Image 55: c
Image 56: c
Image 57: c
Image 58: c
Image 58: o
Image 59: c
Image 60: c
Image 61: c
Image 62: c
Image 63: c
Image 64: c
Image 65: c
Image 66: c
Image 67: c
Image 68: c
Image 69: c
Image 70: c
Image 71: c
Image 72: c
Image 72: o


KeyError: 'list'

In [106]:
labels = {
    'list': roicat.util.JSON_List(labeler.get_labels(kind='list')),
    'df': labeler.get_labels(kind='dataframe'),
}

# Visualize results

In [ ]:
u, c = np.unique(labels['df']['label'], return_counts=True)

plt.figure()
plt.bar(u, c);
plt.xlabel('label');
plt.ylabel('counts');

In [ ]:
roicat.visualization.display_labeled_ROIs(
    images=images,
#     labels=labels['label'].astype(np.int64),
    labels=labels['df'],
    max_images_per_label=10,
    figsize=(10, 3),
    fontsize=25,
    shuffle=False,
)

# Save results

The results file can be opened using any of the following methods:
1. `roicat.helpers.pickle_load(path)`
2. `np.load(path)`
3. 
``` 
import pickle
with open(path_save, mode='rb') as f:
    test = pickle.load(f)
  ```

##### Make and save results

In [109]:
run_data = {
    "data": data.__dict__,
}
results = {
    "labels": labels,
    "images": images,
}

In [ ]:
## Define the directory to save the results to
dir_save = '/media/rich/bigSSD/data_tmp/test_data/'
name_save = 'mouse_1'

paths_save = {
    'labels':   str(Path(dir_save) / f'{name_save}.labeling.labels.csv'),
    'results':  str(Path(dir_save) / f'{name_save}.labeling.results.richfile'),
    'run_data': str(Path(dir_save) / f'{name_save}.labeling.run_data.richfile'),
}

## Save
labels['df'].to_csv(paths_save['labels'])
roicat.util.RichFile_ROICaT(path=paths_save['results']).save(results, overwrite=True)
roicat.util.RichFile_ROICaT(path=paths_save['run_data']).save(run_data, overwrite=True)

# Thank you
If you encountered any difficulties, please let us know at the issues page: https://github.com/RichieHakim/ROICaT/issues

---
# Extras

## Merge and convert GUI CSV files into a results file

##### 1. Make labels dictionary

In [99]:
import csv

# path_csv = r'/home/rich/Desktop/test.csv'  ## Path to the raw output .csv file from the GUI
path_csv = labeler.path_csv

with open(path_csv, 'r') as f:
    lines = np.array([line for line in csv.reader(f)], dtype=str)

labels_dict = {int(idx): label for idx, label in zip(lines[1:, 0], lines[1:, 1])}

##### 2. Make images array

In [ ]:
dir_data = r'/media/rich/bigSSD/downloads_tmp/tmp_data/mouse_0322R/statFiles/20230420/'


path_stat = str(Path(dir_data) / 'stat.npy')
path_ops = str(Path(dir_data) / 'ops.npy')

data = roicat.data_importing.Data_suite2p(
    paths_statFiles=[path_stat],
    paths_opsFiles=[path_ops],
    new_or_old_suite2p='new',
    verbose=True,
)

assert data.check_completeness(verbose=False)['classification_inference'], f"Data object is missing attributes necessary for tracking."

images = np.concatenate(data.ROI_images, axis=0)

##### 3. Make the results dictionary

In [103]:
import pandas as pd

labels_list = ['None'] * len(labels_dict)
for idx, label in labels_dict.items():
    labels_list[idx] = label

labels_df = pd.DataFrame(index=list(labels_dict.keys()), data=labels_list, columns=['label'])

labels = {
    'list': roicat.util.JSON_List(labels_list),
    'dict': roicat.util.JSON_Dict(labels_dict),
    'df': labels_df,
}

In [104]:
run_data = {
    "data": data.__dict__,
}
results = {
    "labels": labels,
    "images": images,
}

##### 4. Save the results dictionary

In [ ]:
## Define the directory to save the results to
dir_save = '/media/rich/bigSSD/data_tmp/test_data/'
name_save = 'mouse_1'

paths_save = {
    'labels':   str(Path(dir_save) / f'{name_save}.labeling.labels.json'),
    'results':  str(Path(dir_save) / f'{name_save}.labeling.results.richfile'),
    'run_data': str(Path(dir_save) / f'{name_save}.labeling.run_data.richfile'),
}

roicat.helpers.json_save(obj=roicat.util.JSON_List(labels['list']), filepath=paths_save['labels'])
roicat.util.RichFile_ROICaT(path=paths_save['results']).save(results, overwrite=True)
roicat.util.RichFile_ROICaT(path=paths_save['run_data']).save(run_data, overwrite=True)